In [20]:
import sqlite3 as sql

In [21]:
def execute_sql_script(file_path, db_path):
    # Connect to the SQLite database
    connection = sql.connect(db_path)

    # Read the SQL script
    with open(file_path, 'r', encoding='ISO-8859-1') as sql_file:
        sql_script = sql_file.read()

    # Execute the script
    cursor = connection.cursor()
    cursor.executescript(sql_script)
    connection.commit()

    # Close the connection
    connection.close()

# Usage
sql_file_path = 'pizzaDB.sql'
db_path = 'pizza.db'
execute_sql_script(sql_file_path, db_path)

In [22]:
con = sql.connect('pizza.db')
cur = con.cursor()

In [23]:
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
table_list

[('poststeder',), ('kunde',), ('pizzatyper',), ('ordre',), ('ordrelinje',)]

# Spørringer

Spørring	1

Finn	kunder	på	postnummer	7000.	Ta	med	navn	og	adresse.

In [24]:
a = [a for a in cur.execute("""
                            SELECT navn, adresse
                            FROM kunde
                            WHERE kunde.postnr = 7000
                            """)]
a

[('Ola', 'Kirkegata 1'), ('Kari', 'Elgesetergate 12'), ('Per', 'Munkegata 10')]

Spørring	2

Hvor	mange	kunder	er	det	i	databasen?	I	resultatet	skal	kolonnen	ha	navnet	
”Antall	kunder”

In [25]:
b = [b for b in cur.execute("""
                            SELECT COUNT(*) AS AntallKunder from kunde
                            """)]
b

[(4,)]

Spørring	3

Finn	alle	poststeder.		Hver	poststed	skal	være	med	bare	en	gang,	og	resultatet	
skal	sorteres	alfabetisk.

In [26]:
c = cur.execute("""
                SELECT DISTINCT poststed
                FROM poststeder
                ORDER BY poststed ASC
                """).fetchall()
c

[('Bodï¿½',), ('Kirkenes',), ('Tromsï¿½',), ('Trondheim',)]

Spørring	4

Skriv	ut	kid,	navn,	adresse,	postnummer	og	poststed	for	alle	kunder

In [27]:
d = cur.execute("""
                SELECT DISTINCT kid, navn, adresse, poststed
                FROM kunde
                INNER JOIN poststeder ON kunde.postnr = poststeder.postnr
                """).fetchall()
d

[(1, 'Ola', 'Kirkegata 1', 'Trondheim'),
 (2, 'Kari', 'Elgesetergate 12', 'Trondheim'),
 (3, 'Per', 'Munkegata 10', 'Trondheim'),
 (4, 'Ola', 'Kongens gate 12', 'Bodï¿½')]

Spørring	5

Hvor	mange	”Thai	Chicken”	er	solgt	og	for	hvor	mange	kroner	til	sammen?

In [28]:
e = cur.execute("""
                SELECT
                    SUM(ordrelinje.antall) as antall,
                    SUM(ordrelinje.delsum) as total
                FROM 
                    ordrelinje
                INNER JOIN
                    pizzatyper on ordrelinje.pizza = pizzatyper.pid
                WHERE
                    pizzatyper.navn = "Thai Chicken"
                """).fetchall()
e

[(3, 762)]

Spørring	6

Finn	alle	pizzatyper	som	noen	har	kjøpt.	Unngår	å	gjenta	pizzanavn	i	resultatet	
og	sorter	resultatet	alfabetisk.

In [29]:
f = cur.execute("""
                SELECT DISTINCT pt.navn
                FROM
                    ordrelinje AS ol
                INNER JOIN
                    pizzatyper AS pt ON ol.pizza = pt.pid
                ORDER BY pt.navn
                """).fetchall()
f

[('Cheese & Tomato',), ('Pepper Steak',), ('Thai Chicken',), ('The Tropical',)]

Spørring	7

Finn	kid,	navn	og	adresse	for	alle	personer	som	bor	på	adresser	som	inneholder	
”gata”.

In [30]:
g = cur.execute("""
                SELECT kid, navn, adresse
                FROM
                    kunde
                WHERE 
                    adresse LIKE '%gata%'
                """).fetchall()
g

[(1, 'Ola', 'Kirkegata 1'), (3, 'Per', 'Munkegata 10')]

Spørring 8

Finn	alle	pizzatyper	som	kunden	”Kari”	ikke	har	kjøpt	ennå.

In [31]:
h = cur.execute("""
                SELECT navn
                FROM
                    pizzatyper AS pt
                WHERE pt.pid NOT IN (
                    SELECT ol.pizza
                    from kunde AS k
                    INNER JOIN ordre AS o ON o.kunde = k.kid
                    INNER JOIN ordrelinje AS ol ON o.ordrenr = ol.ordrenr
                    WHERE
                        k.navn = "Kari"
                )
                """).fetchall()
h
                

[('Heavy Heaven',)]

In [32]:
i = cur.execute("""
                SELECT DISTINCT navn
                FROM kunde as k
                UNION 
                SELECT navn
                FROM pizzatyper AS pt
                """).fetchall()
i

[('Cheese & Tomato',),
 ('Heavy Heaven',),
 ('Kari',),
 ('Ola',),
 ('Pepper Steak',),
 ('Per',),
 ('Thai Chicken',),
 ('The Tropical',)]

Spørring 10

Ta	ut	postnr,	poststed	og	antall	personer	som	bor	på	et	postnummer.	I	resultatet	
skal	alle	postnumre	være	med,	og	det	skal	sorteres	etter	synkende	antall	kunder.

In [33]:
j = cur.execute("""
                SELECT COUNT(k.kid) AS Antall, ps.postnr, ps.poststed
                FROM kunde as k
                RIGHT OUTER JOIN poststeder as ps ON k.postnr = ps.postnr
                GROUP BY ps.postnr
                ORDER BY Antall DESC
                """).fetchall()
j

[(3, '7000', 'Trondheim'),
 (1, '8000', 'Bodï¿½'),
 (0, '9090', 'Kirkenes'),
 (0, '9000', 'Tromsï¿½'),
 (0, '7046', 'Trondheim')]

Spørring 11

Finn	alle	personer	(kid	og	navn)	som	ikke har	kjøpt	”Thai	Chicken”

In [34]:
k = cur.execute("""
                SELECT k.kid, k.navn 
                FROM kunde as k
                WHERE k.kid NOT IN (
                    SELECT o.kunde
                    FROM ordre as o
                    WHERE o.ordrenr IN (
                        SELECT ol.ordrenr
                        FROM ordrelinje AS ol
                            INNER JOIN pizzatyper as pt ON (ol.pizza = pt.pid)
                        WHERE pt.navn = 'Thai Chicken'
                    )
                )
                """).fetchall()
k

[(1, 'Ola'), (4, 'Ola')]

## Innsetting, oppdatering og sletting

Oppgave	1

Kunde	nummer	2,	Kari	,	bestiller	i	dag	(30.	januar	2014)	1	”Pepper	Steak”	og	3	
”Thai	chicken”.	Legg	inn	denne	bestillingen.

In [35]:
# First INSERT
cur.execute("""
    INSERT INTO ordre
    VALUES (6, '2014-01-01', null, 990, 2)
""")

# Second INSERT
cur.execute("""
    INSERT INTO ordrelinje 
    VALUES (6, 1, 3, 1, 228)
""")

# Third INSERT
cur.execute("""
    INSERT INTO ordrelinje 
    VALUES (6, 2, 4, 3, 762)
""")

# Commit the transaction
con.commit()


Oppgave 2

Øk	prisen	på	alle	pizzatyper	med	10	%.

In [36]:
cur.execute("""
            UPDATE pizzatyper
            SET pris = 1.1*pris
            """)
con.commit()

Oppgave 3

Prøv	å	slette	pizzatypen	”Thai	Chicken”.	Hva	skjer?	Hvorfor?	Hva	må	du	evt.	gjøre	
for	å	få	slettet	denne	pizzatypen?

In [37]:
cur.execute("""
            DELETE FROM pizzatyper WHERE navn = 'Thai Chicken'
            """)
con.commit()